<a href="https://colab.research.google.com/github/Moaz-Alhasani/VicunaQA---A-Document-based-Q-A-System-with-Summarization/blob/main/Custom_Website_Chatboot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain langchain-community
!pip -q install pypdf
!pip -q install sentence_transformers
!pip install tokeinzer
!pip install faiss-cpu
!pip install -q unstructured

ERROR: Could not find a version that satisfies the requirement tokeinzer (from versions: none)
ERROR: No matching distribution found for tokeinzer


In [2]:
!pip install nltk==3.91

ERROR: Ignored the following yanked versions: 3.6.4
ERROR: Could not find a version that satisfies the requirement nltk==3.91 (from versions: 2.0b4, 2.0b5, 2.0b6, 2.0b7, 2.0b8, 2.0b9, 2.0.1rc1, 2.0.1rc3, 2.0.1rc4, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.0.5, 3.0.0b1, 3.0.0b2, 3.0.0, 3.0.1, 3.0.2, 3.0.3, 3.0.4, 3.0.5, 3.1, 3.2, 3.2.1, 3.2.2, 3.2.3, 3.2.4, 3.2.5, 3.3.0, 3.4, 3.4.1, 3.4.2, 3.4.3, 3.4.4, 3.4.5, 3.5b1, 3.5, 3.6, 3.6.1, 3.6.2, 3.6.3, 3.6.5, 3.6.6, 3.6.7, 3.7, 3.8, 3.8.1, 3.9b1, 3.9, 3.9.1)
ERROR: No matching distribution found for nltk==3.91


In [3]:
!pip uninstall -y numpy transformers
!pip install numpy transformers

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: transformers 4.49.0
Uninstalling transformers-4.49.0:
  Successfully uninstalled transformers-4.49.0
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured 0.17.0 requires numpy<2, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is inc

In [25]:
import sys
import os
import torch
import textwrap
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain

In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [34]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [35]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [36]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [37]:
text_chunk=text_splitter.split_documents(data)

In [38]:
len(text_chunk)

88

In [39]:
text_chunk[0]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\nSign up\nSign in\nWrite\nSign up\nSign in\nHome\nLibrary\nStories\nStats\nGoPenAI\nHome\nAbout\nFollow publication\nGoPenAI\nWhere the ChatGPT community comes together to share insights and stories.\nFollow publication\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko\nGoPenAI\nAndrew Lukyanenko\nFollow\nPublished in\nGoPenAI\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link')

In [43]:
from langchain.embeddings import HuggingFaceEmbeddings

In [45]:
embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vectorstores=FAISS.from_documents(text_chunk,embedding)

In [51]:
from langchain.llms.base import LLM
from typing import Optional
from pydantic import BaseModel, Field, PrivateAttr

In [53]:
class LLMWapper(LLM, BaseModel):
    max_length: int = Field(100, description="Maximum length of generated text")
    _tokenizer: AutoTokenizer = PrivateAttr()
    _model: AutoModelForSeq2SeqLM = PrivateAttr()

    def __init__(self, tokenizer: AutoTokenizer, model: AutoModelForSeq2SeqLM, **kwargs):
        super().__init__(**kwargs)
        self._tokenizer = tokenizer
        self._model = model

    def _call(self, prompt: str, stop: Optional[list] = None, **kwargs) -> str:
        inputs = self._tokenizer(prompt, return_tensors="pt")
        outputs = self._model.generate(
            input_ids=inputs["input_ids"],
            max_length=self.max_length,
            num_return_sequences=1,
            pad_token_id=self._tokenizer.eos_token_id,
        )
        result = self._tokenizer.decode(outputs[0], skip_special_tokens=True)
        return result

    @property
    def _llm_type(self) -> str:
        return "custom"

In [54]:
llm=LLMWapper(tokenizer=tokenizer, model=model, max_length=100)

In [55]:
llm.predict("please Provide a concise summary of Book Harry potter")
chain=RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstores.as_retriever())

<ipython-input-55-48a79d1b52a6>:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("please Provide a concise summary of Book Harry potter")
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [64]:
result=chain({"question": "How good is vicnna"},return_only_outputs=True)

In [65]:
print(result)

{'answer': '90%* ChatGPT Quality.', 'sources': ''}


In [ ]:
while True:
  query=input(f"Prompt")
  if query =='exit':
    print('Exiting')
    sys.exit()
  if query:
    result=chain({"question": query},return_only_outputs=True)
    print(textwrap.fill(result['answer'],100))


PromptWhat is is stableLM?
Stable LM suite is a suite of language models that allows developers to create and deploy a stable
language model.
PromptCan you please share some details about MPT-7b Model
MPT-7b Model: MosaicML platform detected and addressed 4 hardware failures and resumed the training
run automatically, and - due to architecture and optimization improvements we made - there were no
catastrophic loss spikes.
PromptHow is the best player in NBA
The best player in the NBA is the best player in the NBA.
PromptHow does Llama 2 outperforms other models 
Llama 2's 7 billion and 34 billion parameter models outperform the 7 billion and 40 billion
parameter Falcon models in all benchmark categories
